In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, GridSearchCV ,KFold
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.calibration import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
# df = pd.read_csv('/Users/hneen./iCloud Drive (Archive)/Desktop/L9/Graduation project/colon_predicitions/Data/colon-dataset-processed.csv')
df = pd.read_csv('../Dataset/cleaned_hypertension_data.csv')
df

In [ ]:
X=df.drop('Class',axis=1)
y=df['Class']

In [ ]:

#split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
random_state=123

In [ ]:
#overampling
sm = SMOTE(random_state=random_state)
X_oversampled, y_oversampled = sm.fit_resample(X, y)

X_train_oversampled, X_test_oversampled, y_train_oversampled, y_test_oversampled = train_test_split(X_oversampled, y_oversampled, test_size=0.2, random_state=42)


In [ ]:
#underampling
# rus = RandomUnderSampler(random_state=random_state)
rus = SMOTEENN(random_state=random_state, sampling_strategy='all')
X_undersampled, y_undersampled = rus.fit_resample(X, y)
X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled = train_test_split(X_undersampled, y_undersampled, test_size=0.2, random_state=42)

In [ ]:
n_splits = 5

def GridSer_model(model_params,X_train,y_train):
    scores = []
    cv=KFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    for model_name, mp in model_params.items():
        clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False,n_jobs=-1,error_score=0)
        clf.fit(X_train,y_train)
        best_model = mp['model'].set_params(**clf.best_params_)
        scores.append({
            'best_model': best_model,
            'model': model_name,
            'best_score': clf.best_score_,
            'best_params': clf.best_params_
        })
    return pd.DataFrame(scores,columns=['best_model','model','best_score','best_params']).sort_values(by=['best_score'], ascending=False)

In [ ]:

def model_training(X_train, y_train, X_test,  y_test, models):
    
    # Create an empty DataFrame to store results
    results_df = pd.DataFrame(columns=['Model', 'Accuracy'])

    for model in models:
        
        model.fit(X_train, y_train)
        # Make predictions
        predictions = model.predict(X_test)
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, predictions)
        precision =precision_score(y_test, predictions)
        recall= recall_score(y_test, predictions)
        cm=confusion_matrix(y_test, predictions)
        labels=model.classes_
        
        # Append results to the DataFrame
        temp_df = pd.DataFrame({'Model': [model.__class__.__name__], 'Accuracy': [accuracy], 'Precision': [precision], 'recall': [recall], 'cm':[cm],'labels':[labels]})
        results_df = pd.concat([results_df, temp_df], ignore_index=True)


    # Sort and return the results DataFrame
    return results_df.sort_values(by=['Accuracy'], ascending=False)


In [ ]:
model_params = {
    
    'XGBClassifier' : {
        'model': XGBClassifier(),
        'params': {
            'n_estimators': [50, 100, 200], 
            'learning_rate': [0.01, 0.1, 1],
            'max_depth': [3, 5, 10],
            'gamma': [0, 0.1, 0.5, 1],
            'subsample': [0.5, 0.75, 1],
            'colsample_bytree': [0.5, 0.75, 1],
            'lambda': [0.0, 0.1, 1.0]
        }
    },
    
    'KNeighborsClassifier': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 10, 15],  # Number of neighbors to use
            'weights': ['uniform', 'distance'],  # Weight function used in prediction
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute the nearest neighbors
            'leaf_size': [10, 30, 50, 70],  # Leaf size passed to BallTree or KDTree
            'p': [1, 2]  # Power parameter for the Minkowski metric
        }
    },
    
    'SVM':{
            'model': SVC(),
            'params': {
                'C': [0.1, 1, 10, 100], 
                'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
            }
    },  
    
    'ExtraTreesClassifier' : {
        'model': ExtraTreesClassifier(),
        'params': {
        'n_estimators': [50, 100, 200],
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 4, 6],
        'min_samples_leaf': [1, 2, 3],
        'max_features': ['sqrt', 'log2']
        }
    },    
    
    'HistGradientBoostingClassifier':{
        'model': HistGradientBoostingClassifier(),
        'params': {
            'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
            'max_iter': [50, 100, 200],
            'max_leaf_nodes': [15, 31, 63, 127],
            'min_samples_leaf': [5, 10, 20],
            'l2_regularization': [0.0, 0.1, 1.0]
        }
    },
    
    'RandomForestClassifier' : {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 4, 6],
            'min_samples_leaf': [1, 2, 3],
            'max_features': ['auto', 'sqrt', 'log2']
        }
    },
    
    'DecisionTreeClassifier':{
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['auto', 'sqrt', 'log2', None]
        }
    },
    
    'AdaBoostClassifier' : {
        'model': AdaBoostClassifier(),
        'params': {
            'n_estimators': [50, 100, 200], 
            'learning_rate': [0.01, 0.1, 1]
        }
    } ,

    'LogisticRegression' : {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10, 100], 
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
        }
    }
    
}

In [ ]:
original = GridSer_model(model_params,X_train,y_train)

In [ ]:
original

In [ ]:
original['best_params'][0]

In [ ]:
models=original['best_model'].tolist()
model_training(X_train, y_train, X_test, y_test, models)


In [ ]:
oversampled_grd=GridSer_model(model_params,X_train_oversampled,y_train_oversampled)

In [ ]:
oversampled_grd

In [ ]:
models_oversampled_grd=oversampled_grd['best_model'].tolist()
model_training(X_train_oversampled, y_train_oversampled, X_test_oversampled, y_test_oversampled, models_oversampled_grd)

In [ ]:
undersampled_grd=GridSer_model(model_params,X_train_undersampled,y_train_undersampled)

In [ ]:
undersampled_grd

In [ ]:
undersampled_grd['best_params'][1]

In [ ]:
models_undersampled_grd=undersampled_grd['best_model'].tolist()
models_undersampled_grd_t=model_training(X_train_undersampled, y_train_undersampled, X_test_undersampled, y_test_undersampled, models_undersampled_grd)

In [ ]:
models_undersampled_grd_t


In [ ]:
cm=models_undersampled_grd_t['cm'][0]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot(cmap=plt.cm.Blues)
plt.title('AdaBoostClassifier')
plt.show()

In [ ]:
# models_undersampled_grd_t
cm=models_undersampled_grd_t['cm'][1]
cm
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot(cmap=plt.cm.Blues)
plt.title('HistGradientBoostingClassifier')
plt.show()

In [ ]:
# models_undersampled_grd_t
cm=models_undersampled_grd_t['cm'][2]
cm
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot(cmap=plt.cm.Blues)
plt.title('ExtraTreesClassifier')
plt.show()

In [ ]:
# import pickle
# pickle.dump(models_undersampled_grd[0], open('/Users/shahadaleissa/Downloads/Code/SMOTEENN_Models/Adaboost_SMOTEENN.pkl','wb'))
# pickle.dump(models_undersampled_grd[2], open('/Users/shahadaleissa/Downloads/Code/SMOTEENN_Models/ET_SMOTEENN.pkl','wb'))
# pickle.dump(models_undersampled_grd[1], open('/Users/shahadaleissa/Downloads/Code/SMOTEENN_Models/HGB_SMOTEENN.pkl','wb'))